In [13]:
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
import pylab
from matplotlib.colors import LinearSegmentedColormap
import SG
from scipy import interpolate
import Analysis

In [4]:
%matplotlib

Using matplotlib backend: TkAgg


In [5]:
data2011 = np.load('2011_Prune_strong_flat.npy')
data2014 = np.load('2014_Prune_strong.npy')

d2011 = list()
for i in range(len(data2011)):
    d2011.append(np.array([tuple(j) for j in data2011[i]],dtype=np.float))

d2014 = list()
for i in range(len(data2014)):
    d2014.append(np.array([tuple(j) for j in data2014[i]],dtype=np.float))

In [31]:
def phase_switch(data,minus=0):
    temp = list()
    temp.append(np.array(data[0]))
    flip = True
    for i in range(1,len(data)):
        temp.append(np.array(data[i]))
        if data[i-1][1] == data[i][1] and data[i-1][2] == data[i][2]:
            if flip:
                temp[-1][2] = 2*(1 - minus) - temp[-1][2]
                flip = False
            else:
                flip = True
        else:
            flip = True
    return np.array(temp)

In [32]:
#Using most recent 2019 fit points
pruned14 = np.load('Pruned 025 4_16_2019.npy')
pruned34 = np.load('Pruned 075 2_1_2019.npy')
pruned12 = np.load('Pruned 05 3_12_2019.npy')
prunedfig = np.load('Pruned 1 Fig 3_12_2019.npy')
prunedsup = np.load('Pruned 1 Sup 3_12_2019.npy')

new14 = list()
for i in pruned14:
    for j in i:
        new14.append(j)
new14 = np.array(new14)

new34 = list()
for i in pruned34:
    for j in i:
        new34.append(j)
new34 = np.array(new34)

new12 = list()
for i in pruned12:
    for j in i:
        new12.append(j)
new12 = np.array(new12)


newfig = list()
for i in prunedfig:
    for j in i:
        newfig.append(j)
newfig = np.array(newfig)

newsup = list()
for i in prunedsup:
    for j in i:
        newsup.append(j)
        
newsup = np.array(newsup)

temp = list(new14)
pi14 = np.array(temp.pop(-3))
new14 = np.array(temp)
pi = np.vstack((newsup,newfig,pi14))


sw14m = phase_switch(new14,minus = 1)
pim = phase_switch(pi,minus = 1)


In [6]:
plt.rc('text', usetex=True) 

# Helvetica 
# set serif, too. Otherwise setting to times and then 
# Helvetica causes an error. 
font = {'family':'sans-serif','sans-serif':['Helvetica'], 
        'serif':['cm10']} 
preamble  = r""" 
               \usepackage{color} 
               \usepackage[tx]{sfmath} 
               \usepackage{helvet} 
            """ 


# Unicode for Tex 
#preamble =  r"""\usepackage[utf8]{inputenc}""" + preamble 
# inputenc should be set automatically 
plt.rcParams['text.latex.unicode']=True 

#print font, preamble 
plt.rc('font',**font) 
plt.rcParams['text.latex.preamble'] = preamble
mpl.rcParams.update({'font.size': 11})

In [16]:
cdict = {'red':   ((0.0, 0.0, 0.0),
                   (0.5, 1.0, 1.0),
                   (1.0, 1.0, 1.0)),

         'green': ((0.0, 0.0, 0.0),
                   (0.5,0.0,0.0),
                   (1.0, 1.0, 1.0)),

         'blue':  ((0.0, 1.0, 1.0),
                   (0.5, 0.0, 0.0),
                   (1.0, 0.0, 0.0))
        }
        
yellowredblue = LinearSegmentedColormap('YellowRedBlue', cdict)

In [9]:
def percentile_normalize(data,p=1):
    minv = np.percentile(data,p)
    maxv = np.percentile(data,100-p)
    hold = np.array(data)
    for i in range(len(hold)):
        hold[i] = (hold[i]-minv)/(maxv-minv)
    return hold

In [10]:
def search(data, t):
    seq = data[0]    
    min = 0
    max = len(seq) - 1
    while True:        
        m = (min + max) // 2
        if max < min:
            if np.abs(t-seq[m]) < np.abs(t-seq[m-1]):          
                return m
            else:
                return m-1        
        if seq[m] < t:
            min = m + 1
        elif seq[m] > t:
            max = m - 1
        else:
            return m
            

def clip(d,min,max):
    data = np.array(d)
    li = search(data,min)
    ri = search(data,max)
    return data[:,li:ri+1]
    

In [11]:
def normalize(data):
    minv = min(data)
    maxv = max(data)
    hold = np.array(data)
    for i in range(len(hold)):
        hold[i] = (hold[i]-minv)/(maxv-minv)
    return hold

# 2014 Data

In [98]:
#More smoothing, better option
bfieldval = Analysis.bval2014
n = 1024
smoothval = 51
poly = 2
interpdim = 2
data = d2014
cmlabel = 'Normalized $\Delta$R$_D$ (k$\Omega$)'
data = [x for (y,x) in sorted(zip(bfieldval,data), key=lambda pair: pair[0])]
bval = sorted(bfieldval)    
xmin = -100
xmax = 0
for i in data:
    if i[0][0] > xmin:
        xmin = i[0][0]
    if i[0][-1] < xmax:
        xmax = i[0][-1]
#xmin += 0.01
#xmax -= 0.01
xv = np.linspace(xmin,xmax,n)
intv = list()
for i in data:
    #Try flipping filtering and interp
    dataclip = clip(i,xmin,xmax)
    dataclip[1] = SG.savitzky_golay(dataclip[1],smoothval,poly)
    tck = interpolate.splrep(dataclip[0], dataclip[1],k=1)
    y = interpolate.splev(xv,tck)
    intv.append(percentile_normalize(y,1))
xb = np.linspace(bval[0],bval[-1],n)
b = list()
for i in range(len(intv[0])):
    hold = list()
    for j in range(len(intv)):
        hold.append(intv[j][i])
    b.append(hold)
intb = list()
for i in b:
    tck = interpolate.splrep(bval,i,k=interpdim)
    y = interpolate.splev(xb,tck)
    intb.append(y)
Z = np.array(intb)
X,Y = pylab.meshgrid(xb,xv)

In [75]:
colortype = yellowredblue
f, ax = plt.subplots()
plt.pcolormesh(X,Y,Z,vmin=0,vmax=1,cmap=colortype)
plt.ylim(xmin,xmax)
plt.xlim(bval[0],bval[-1])
plt.ylabel('Gate Voltage (mV)',size=18)
plt.xlabel('B Field (T)',size=18)
cbar = plt.colorbar()
cbar.set_label(cmlabel,size=14)
ax.get_xaxis().get_major_formatter().set_useOffset(False)

plt.show()

In [99]:
#Gray Scale
f, ax = plt.subplots()
plt.pcolormesh(X,Y,Z,vmin=0,vmax=1,cmap='binary')
plt.ylim(xmin,xmax)
plt.xlim(bval[0],bval[-1])
plt.ylabel('Gate Voltage(mV)',size=18)
plt.xlabel('B Field(T)',size=18)
cbar = plt.colorbar()
cbar.set_label(cmlabel,size=14)
ax.get_xaxis().get_major_formatter().set_useOffset(False)

In [90]:
#Markers
shift = 0.00008
for i in sorted(new14,key = lambda x: x[0]):
    if i[1]>Analysis.bval2011[0]:
        x = float(i[1])
        if x==5.5845:
            x -= shift
        q = plt.scatter(x,i[0],s=70,color='#00FFFF',marker='d',edgecolor='k',linewidth='1',label='$\Delta\\theta=\pi/4')

for i in pi:
    if i[1]>Analysis.bval2011[0]:
        x = float(i[1])
        if x==5.5845:
            x -= shift
        p = plt.scatter(x,i[0],s=60,color='r',edgecolor='k',linewidth='1',label='$\Delta\\theta=\pi')

plt.legend([q,p], ["$\Delta\\theta =\pi/4$","$\Delta\\theta =\pi$"],loc=2,scatterpoints=1)

In [84]:
f.savefig('Grayscale Color Map with Scatter 14 v2',dpi=1200)

In [104]:
#Markers with +/- signs
shift = 0.00008
for i in sw14m:
    if i[1]>Analysis.bval2011[0]:
        x = float(i[1])
        if x==5.5845:
            x -= shift
        if i[2] > 0:
            q = plt.scatter(x,i[0],s=70,color='#00FFFF',marker='d',edgecolor='k',linewidth='1',label='$\Delta\\theta=\pi/4')
        else:
            qm = plt.scatter(x,i[0],s=70,color='#00FF00',marker='d',edgecolor='k',linewidth='1',label='$\Delta\\theta=-\pi/4')

for i in pim:    
    if i[1]>Analysis.bval2011[0]:
        x = float(i[1])
        if x==5.5845:
            x -= shift
        if i[2] > 0:
            p = plt.scatter(x,i[0],s=60,color='r',edgecolor='k',linewidth='1',label='$\Delta\\theta=\pi')
        else:
            pm = plt.scatter(x,i[0],s=60,color='orange',edgecolor='k',linewidth='1',label='$\Delta\\theta=-\pi')

plt.legend([q,qm,p,pm], ["$\Delta\\theta =\pi/4$","$\Delta\\theta =-\pi/4$","$\Delta\\theta =\pi$","$\Delta\\theta =-\pi$"],loc=2,scatterpoints=1)

In [105]:
f.savefig('Grayscale Color Map with Scatter 14 v3 PM',dpi=1200)

In [91]:
new14

array([[-30.9763    ,   5.5805    ,   0.35446229,   1.        ],
       [-30.11417   ,   5.5805    ,   0.35446229,   1.        ],
       [-28.8464    ,   5.5805    ,   0.35446229,   1.        ],
       [-26.6193    ,   5.5805    ,   0.35446229,   1.        ],
       [-27.5842    ,   5.5805    ,   0.35446229,   1.        ],
       [-25.1515    ,   5.5805    ,   0.25995879,   1.        ],
       [-25.007     ,   5.5805    ,   0.25995879,   1.        ],
       [-21.99253   ,   5.581     ,   0.24561601,   1.        ],
       [-21.041845  ,   5.581     ,   0.24561601,   1.        ],
       [-20.24955   ,   5.583     ,   0.25      ,   1.        ],
       [-19.38313   ,   5.583     ,   0.25      ,   1.        ],
       [-19.18627   ,   5.583     ,   0.25      ,   1.        ],
       [-24.267575  ,   5.583     ,   0.27320272,   1.        ],
       [-19.4861    ,   5.5835    ,   0.27931602,   1.        ],
       [-18.62333   ,   5.5835    ,   0.28183099,   1.        ],
       [-18.2212    ,   5

# 2011 Data

In [106]:
bfieldval = Analysis.bval2011
n = 1024
smoothval = 51
poly = 2
interpdim = 2
data = d2011
cmlabel = 'Normalized $\Delta$R$_D$ (k$\Omega$)'
data = [x for (y,x) in sorted(zip(bfieldval,data), key=lambda pair: pair[0])]
bval = sorted(bfieldval)    
xmin = None
xmax = 0
for i in data:
    if i[0][0] > xmin:
        xmin = i[0][0]
    if i[0][-1] < xmax:
        xmax = i[0][-1]
xmin += 0.5
xmax -= 0.5
xv = np.linspace(xmin,xmax,n)
intv = list()
count = 0
for i in data:
    if count == 21:
        xmin += 1
    #Try flipping filtering and interp
    dataclip = clip(i,xmin,xmax)
    dataclip[1] = SG.savitzky_golay(dataclip[1],smoothval,poly)
    tck = interpolate.splrep(dataclip[0], dataclip[1],k=1)
    y = interpolate.splev(xv,tck)
    intv.append(percentile_normalize(y,1))
    count += 1
xb = np.linspace(bval[0],bval[-1],n)
b = list()
for i in range(len(intv[0])):
    hold = list()
    for j in range(len(intv)):
        hold.append(intv[j][i])
    b.append(hold)
intb = list()
for i in b:
    tck = interpolate.splrep(bval,i,k=interpdim)
    y = interpolate.splev(xb,tck)
    intb.append(y)
Z = np.array(intb)
X,Y = pylab.meshgrid(xb,xv)


In [53]:
colortype = yellowredblue
f, ax = plt.subplots()
plt.pcolormesh(X,Y,Z,vmin=0,vmax=1,cmap=colortype)
plt.ylim(xmin,xmax)
plt.xlim(bval[0],bval[-1])
plt.ylabel('Gate Voltage (mV)',size=18)
plt.xlabel('B Field (T)',size=18)
cbar = plt.colorbar()
cbar.set_label(cmlabel,size=14)
ax.get_xaxis().get_major_formatter().set_useOffset(False)

plt.show()

In [108]:
#Gray Scale
f, ax = plt.subplots()
plt.pcolormesh(X,Y,Z,vmin=0,vmax=1,cmap='binary')
plt.ylim(xmin,xmax)
plt.xlim(bval[0],bval[-1])
plt.ylabel('Gate Voltage(mV)',size=18)
plt.xlabel('B Field(T)',size=18)
cbar = plt.colorbar()
cbar.set_label(cmlabel,size=14)
ax.get_xaxis().get_major_formatter().set_useOffset(False)

In [94]:
#Markers
#for i in new14:
#    if i[1]<=bval2011[0]:
#        q = plt.scatter(i[1],i[0],s=60,color='#00FFFF',marker='d',edgecolor='k',linewidth='1',label='$\Delta\phi=\pi/4')
for i in pi:    
    if i[1]<=Analysis.bval2011[0]:
        p = plt.scatter(i[1],i[0],s=60,color='r',edgecolor='k',linewidth='1',label='$\Delta\theta=\pi')
plt.legend([p], ["$\Delta\\theta =\pi$"],loc=2,scatterpoints=1)

In [109]:
#Markers PM
#for i in new14:
#    if i[1]<=bval2011[0]:
#        q = plt.scatter(i[1],i[0],s=60,color='#00FFFF',marker='d',edgecolor='k',linewidth='1',label='$\Delta\phi=\pi/4')
for i in pim:
    if i[1]<=Analysis.bval2011[0]:
        if i[2] > 0:
            p = plt.scatter(i[1],i[0],s=70,color='r',edgecolor='k',linewidth='1',label='$\Delta\theta=\pi')
        else:
            pm = plt.scatter(i[1],i[0],s=70,color='orange',edgecolor='k',linewidth='1',label='$\Delta\theta=-\pi')
plt.legend([p,pm], ["$\Delta\\theta =\pi$","$\Delta\\theta =-\pi$"],loc=2,scatterpoints=1)

In [110]:
f.savefig('Grayscale Color Map with Scatter 11 v3 PM',dpi=1200)

In [111]:
new12

array([[-24.66637   ,   5.5265    ,   0.47590564,   0.8836773 ],
       [-30.623075  ,   5.534     ,   0.532419  ,   0.80487805],
       [-28.549115  ,   5.534     ,   0.532419  ,   0.80487805],
       [-28.18076   ,   5.534     ,   0.532419  ,   0.80487805],
       [-27.62813   ,   5.534     ,   0.532419  ,   0.80487805],
       [-27.450285  ,   5.534     ,   0.532419  ,   0.80487805]])

In [112]:
new34

array([[-35.4633    ,   5.5355    ,   0.75090375,   0.91954023],
       [ -6.1461205 ,   5.583     ,   0.8       ,   1.        ],
       [-34.79144   ,   5.531     ,   0.79002495,   0.90654206],
       [ -9.12448   ,   5.5835    ,   0.71595502,   1.        ],
       [-37.5528    ,   5.536     ,   0.79171949,   0.90690691],
       [-37.19396   ,   5.536     ,   0.79171949,   0.90690691],
       [-36.85785   ,   5.536     ,   0.79171949,   0.90690691],
       [-36.437425  ,   5.536     ,   0.79171949,   0.90690691],
       [-36.142725  ,   5.536     ,   0.79171949,   0.90690691],
       [-34.101725  ,   5.5295    ,   0.78772907,   0.91935484]])